In [1]:
import pandas as pd
import numpy as np
import re
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
import joblib
from datetime import datetime

# Custom Tokenizer
def custom_tokenizer(text):
    if pd.isna(text):
        return []
    tokens = re.findall(r'\w+', str(text).lower())
    return [subtok for token in tokens for subtok in re.findall(r'[a-z]+|\d+', token)]

# Custom TfidfVectorizer
class CustomTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        return lambda doc: custom_tokenizer(doc)

def clean_columns(df):

    df['index'] = df['index'].astype(str)
    # Extract the ID from the column name 
    df['ID'] = df['Original Column'].str.extract(r'(\d+)\.')
    
    # Extract the Column name up to the first colon, slash, or parenthesis
    df['Column'] = df['Original Column'].str.extract(r'\d+\.\s*([^/(:]+)')
    
    # Extract everything after the first colon, slash, or parenthesis as the full description
    df['Description'] = df['Original Column'].str.extract(r'\d+\.\s*[^/(:]+\s*[:(/](.*)')[0]
    
    # For cases where Description is None (like the new case with parentheses),
    # extract the content within parentheses
    mask = df['Description'].isnull()
    df.loc[mask, 'Description'] = df.loc[mask, 'Original Column'].str.extract(r'\(([^)]+)\)')
    
    # Clean up: Trim spaces from all string columns
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # Replace empty strings with None
    df = df.replace(r'^\s*$', None, regex=True)
    
    # Remove any remaining parentheses from Description and add space between type and size
    df['Description'] = df['Description'].str.replace(r'\(', ' ', regex=True).str.replace(r'\)', '', regex=True)
    
    # Reorder columns to ensure ID comes before Column
    columns_order = ['index', 'name', 'area', 'Original Column', 'ID', 'Column', 'Description']
    df = df.reindex(columns=columns_order)
    
    return df

def replace_abbreviations(text, abbreviations_dict):
    words = text.split()
    replaced_words = [abbreviations_dict.get(word, word) for word in words]
    return " ".join(replaced_words)

def preprocess_columns(df, abbreviations_dict):
    # Only convert "Column" to string type for non-null values and then to lowercase to create "CleanedColumn"
    mask = df['Column'].notna()
    df.loc[mask, 'CleanedColumn'] = df.loc[mask, 'Column'].astype(str).apply(split_camel_case)
    
    # Remove '-' and '_' characters only for non-null values
    mask = df['CleanedColumn'].notna()
    df.loc[mask, 'CleanedColumn'] = df.loc[mask, 'CleanedColumn'].str.replace('[-_]', ' ', regex=True)

    # Apply abbreviations dictionary to Description
    mask = df['Description'].notna()
    df.loc[mask, 'Description'] = df.loc[mask, 'Description'].apply(lambda x: replace_abbreviations(x, abbreviations_dict))
 
    return df

def split_camel_case(s):
    # Special case for 'pH'
    if s.lower() == 'ph':
        return 'ph'
    
    # Split the string into words
    words = re.findall(r'[A-Z]{2,}(?=[A-Z][a-z]+[0-9]*|\d|\W|$)|\w+|[^\w\s]', s)
    
    # Process each word
    processed_words = []
    for word in words:
        # Keep acronyms and all-uppercase words as is
        if word.isupper() and len(word) > 1:
            processed_words.append(word)
        else:
            # Split camel case for other words
            split_word = re.findall(r'[A-Z]?[a-z]+|[A-Z]{2,}(?=[A-Z][a-z]|\d|\W|$)|\d+', word)
            processed_words.extend(split_word)
    
    # Join the words and convert to lowercase
    return ' '.join(processed_words).lower()

def apply_analysis(df, target_words_dict, description_words_dict, abbreviations_dict, all_datasets_info, DB_or_dataset):

    def get_special_name_format(table_name, column_name):
        special_cases = {
            'city': 'city',
            'country': 'country',
            'state': 'state',
            'province': 'state'  # province is treated as state
        }

        if DB_or_dataset.lower() == 'd':
            # For datasets, table_name is a number, so we don't check it
            if column_name.lower() == 'name':
                return 'name', target_words_dict['name']
        else:
            # For database tables, check if any special case is in the table name
            table_name_str = str(table_name).lower()
            for case, format_type in special_cases.items():
                if case in table_name_str and column_name.lower() == 'name':
                    return f"{format_type}_name", target_words_dict[format_type]
        
        return 'name', target_words_dict['name']
    
    # main code 
    # Check if PK/FK information is available
    has_pk_fk_info = 'primary_key' in all_datasets_info.columns and 'foreign_keys' in all_datasets_info.columns

    # Get primary and foreign keys for each table
    pk_fk_dict = {}
    if has_pk_fk_info:
        for _, row in all_datasets_info.iterrows():
            table_name = row['index']
            pk = row['primary_key'] if pd.notna(row['primary_key']) else None
            fks = row['foreign_keys'].split(', ') if pd.notna(row['foreign_keys']) else []
            fk_columns = [fk.split(' -> ')[0] for fk in fks]
            pk_fk_dict[table_name] = {'pk': pk, 'fks': fk_columns} 

    # 'formats_ordered_list' is a list of dictionary keys in the order they appear in 'formats_dictionary'
    formats_ordered_list = list(target_words_dict.keys())

    # Initialize new columns
    df['ColumnKeyword'] = None
    df['ColumnFormat'] = None
    df['DescriptionKeyword'] = None
    df['DescriptionFormat'] = None
   
    # Apply target words analysis
    for i, row in df.iterrows():
        std_col_name = row['CleanedColumn']
        col_name = row['Column']
        table_name = row['index']

        # Skip if 'CleanedColumn' is missing
        if pd.isnull(std_col_name):
            continue

        # Split camel case
        std_col_name = split_camel_case(std_col_name)

        # Check if the column is a primary key or foreign key
        if table_name in pk_fk_dict:
            if col_name == pk_fk_dict[table_name]['pk'] or col_name in pk_fk_dict[table_name]['fks']:
                df.at[i, 'ColumnKeyword'] = 'id'
                df.at[i, 'ColumnFormat'] = 'IDcolumn'
                continue

        found = False

        # Iterate through each word based on the order in 'formats_ordered_list'
        for word in formats_ordered_list:
            analysis = target_words_dict[word]
            # Special handling for 'name'
            if word == 'name':
                keyword, format_type = get_special_name_format(table_name, col_name)
                if keyword != 'name':
                    df.at[i, 'ColumnKeyword'] = keyword
                    df.at[i, 'ColumnFormat'] = format_type
                    found = True
                    break
                pattern = rf'({word})(?![\\w-])'
            # Special handling for uppercase 'ID' at the end of a column name
            elif word == 'id' and col_name.endswith('ID'):
                df.at[i, 'ColumnKeyword'] = 'id'
                df.at[i, 'ColumnFormat'] = target_words_dict.get('id', 'ID column')
                found = True
                break
            else:
                # General matching for other terms
                pattern = rf'\b{word}\b'

            # Search for the pattern in the CleanedColumn
            if re.search(pattern, std_col_name, re.IGNORECASE):
                df.at[i, 'ColumnKeyword'] = word
                df.at[i, 'ColumnFormat'] = analysis
                found = True
                break

        # If no match found, replace abbreviations and try again
        if not found:
            replaced_text = replace_abbreviations(std_col_name, abbreviations_dict)
            for word in formats_ordered_list:
                pattern = rf'\b{word}\b'
                if re.search(pattern, replaced_text, re.IGNORECASE):
                    df.at[i, 'ColumnKeyword'] = word
                    df.at[i, 'ColumnFormat'] = target_words_dict[word]
                    found = True
                    break

        # If still no match, break down the word into substrings
        if not found:
            for j in range(len(row['CleanedColumn']), 2, -1):
                for k in range(len(row['CleanedColumn']) - j + 1):
                    subword = row['CleanedColumn'][k:k+j]
                    
                    # Check if the subword exists in the abbreviation dictionary
                    expanded_subword = abbreviations_dict.get(subword, None)
                    if expanded_subword:
                        # If the expanded subword exists in the target words dictionary, use it
                        if expanded_subword in target_words_dict:
                            df.at[i, 'ColumnKeyword'] = expanded_subword
                            df.at[i, 'ColumnFormat'] = target_words_dict[expanded_subword]
                            found = True
                            break
                        
                    # Else, continue with the original subword
                    elif subword in target_words_dict:
                        df.at[i, 'ColumnKeyword'] = subword
                        df.at[i, 'ColumnFormat'] = target_words_dict[subword]
                        break

                if found:
                    break

    # Apply description words analysis
    for i, row in df.iterrows():
        # Skip if 'Description' is missing
        if pd.isnull(row['Description']):
            continue

        for word, analysis in description_words_dict.items():
            if ((not pd.isnull(row['Description']) and re.search(rf'\b{word}\b', row['Description'], re.IGNORECASE))):
                df.at[i, 'DescriptionKeyword'] = word
                df.at[i, 'DescriptionFormat'] = analysis
                break

    return df

def get_top_features(instance, feature_names, clf, top_n=3):
    feature_importances = sorted(
        [(importance, name) for importance, name in zip(instance.toarray()[0], feature_names) if importance > 0],
        reverse=True
    )[:top_n]
    return [(name, importance) for importance, name in feature_importances]

def custom_resample(X, y, min_samples=6):
    # Convert to CSR format if it's not already
    if not isinstance(X, sp.csr_matrix):
        X = X.tocsr()

    class_counts = Counter(y)
    X_resampled = []
    y_resampled = []
    
    for class_label, count in class_counts.items():
        class_indices = np.where(y == class_label)[0]
        if count < min_samples:
            n_samples = min_samples
            resampled_indices = np.random.choice(class_indices, size=n_samples, replace=True)
        else:
            n_samples = count
            resampled_indices = class_indices
        
        X_resampled.append(X[resampled_indices])
        y_resampled.extend([class_label] * n_samples)
    
    X_resampled = sp.vstack(X_resampled)
    return X_resampled, np.array(y_resampled)

def efficient_smote(X, y, sampling_strategy='auto', k_neighbors=5):
    # Convert to CSR format if it's not already
    if not isinstance(X, sp.csr_matrix):
        X = X.tocsr()

    X_resampled, y_resampled = custom_resample(X, y)

    # Create a NearestNeighbors estimator
    nn = NearestNeighbors(n_neighbors=k_neighbors + 1, n_jobs=-1)
    
    # Initialize SMOTE with the NearestNeighbors estimator
    smote = SMOTE(sampling_strategy=sampling_strategy, k_neighbors=nn, n_jobs=None)
    
    # Apply SMOTE
    X_resampled, y_resampled = smote.fit_resample(X_resampled, y_resampled)
    
    return X_resampled, y_resampled

def train_evaluate_save_model(model, model_name, X_train, X_test, y_train, y_test, df, vectorizer, weights, feature_names , output_file_path_text):
    print(f"\nTraining {model_name}...")
    
    param_grid = {
        'RandomForestClassifier': {
            'n_estimators': [100, 200],
            'max_depth': [None, 30],
            'min_samples_split': [2, 5],
        },
        'LogisticRegression': {
            'C': [0.1, 1],
            'solver': ['lbfgs'],
            'max_iter': [20000],
            'warm_start': [True]
        },
        'GradientBoostingClassifier': {
            'n_estimators': [100],
            'learning_rate': [0.1],
            'max_depth': [3],
        },
        'KNeighborsClassifier': {
            'n_neighbors': [3, 5],
            'weights': ['uniform', 'distance'],
        },
        'LinearSVC': {
            'C': [0.1, 1],
            'max_iter': [20000],
            'dual': ['auto']
        }
    }

    grid_search = GridSearchCV(model, param_grid[model_name], cv=3, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")

    y_pred = best_model.predict(X_test)
    report = classification_report(y_test, y_pred, zero_division=1)
    print(f"Classification Report for {model_name}:")
    print(report)

    # Save the model
    joblib.dump(best_model, f'{model_name}_model.joblib')

    # Create predictions for all data
    X_full = vectorizer.transform(df['combined'])
    if sp.issparse(weights):
        X_full = X_full.multiply(weights.diagonal())
    else:
        X_full = X_full.multiply(weights)
    
    df[f'predicted_format_{model_name}'] = best_model.predict(X_full)
    
    # Calculate top features
    df[f'top_features_{model_name}'] = df.apply(lambda row: get_top_features(
        vectorizer.transform([row['combined']]).multiply(weights), feature_names, best_model), axis=1)

    # Add DIF column
    def compare_formats(actual, predicted, output_file_path_text):
        if actual in ['numerical', 'numerical>=0'] and predicted in ['numerical', 'numerical>=0']:
            return 0
        return int(actual != predicted)

    # Add DIF column
    df[f'DIF_{model_name}'] = df.apply(lambda row: compare_formats(row['FinalFormat'], row[f'predicted_format_{model_name}'], output_file_path_text), axis=1)

    # Sort the DataFrame
    df_sorted = df.sort_values(by=[f'DIF_{model_name}', f'predicted_format_{model_name}', 'FinalFormat'], ascending=[False, True, True])

    # Save the DataFrame to an Excel file
    #output_file_path = f'AnalysedColumnsDB_with_Predictions_{model_name}.xlsx'
    output_file_path = f'{output_file_path_text}_{model_name}.xlsx'
    df_sorted.to_excel(output_file_path, index=False)
    print(f"DataFrame for {model_name} saved to {output_file_path}")

    return best_model, report

def main():
    print(f"It started on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    DB_or_dataset = input("Insert if using Datasets (d) or Database.tables (dt) or All Data Sources (ds): ")

    print(DB_or_dataset)

    if DB_or_dataset.lower() == 'd':
        print("Using Datasets")
        datasets_xlsx = pd.read_excel("FiftyDatasets.xlsx") 
        columns_xlsx = "AllColumnsFromFiftyDatasets.xlsx"
        analysed_columns_file_path = 'AnalysedColumns.xlsx'     
        output_file_path_text = 'AnalysedColumns_with_Predictions'
    elif DB_or_dataset.lower() == 'dt':
        print("Using Database tables")
        datasets_xlsx = pd.read_excel("AllDatasetsInfo.xlsx")
        columns_xlsx = "AllColumnsInfo.xlsx"
        analysed_columns_file_path = 'AnalysedColumnsDB.xlsx'
        output_file_path_text = 'AnalysedColumnsDB_with_Predictions'
    elif DB_or_dataset.lower() == 'ds':
        print("Using Datasets and Database tables")
        datasets_xlsx = pd.read_excel("AllDatasourcessInfo.xlsx")
        columns_xlsx = "AllAttributes_andColumnsInfo.xlsx"
        analysed_columns_file_path = 'AnalysedColumnsDS.xlsx'
        output_file_path_text = 'AnalysedColumnsDS_with_Predictions'
    else:
        raise ValueError("Invalid input. Please enter 'd' for Datasets or 'dt' for Database tables or or 'ds' for All Data Sources.")

    print(datasets_xlsx.head())

    # Load the dataset
    df = pd.read_excel(columns_xlsx)
    print(f"Original df shape: {df.shape}")

    # Load dictionaries
    dictionary = {}

    # Open the formats dictionary file and read line by line 
    with open("formats_dictionary.txt", "r") as file:
        for line in file:
            # Remove the trailing newline and comma, then split the line into key and value at the colon
            key, value = line.rstrip(",\n").split(":")
        
            # Remove the quotes around the key and value
            key = key.strip("'")
            value = value.strip("'")

            # Add the key-value pair to the dictionary
            dictionary[key] = value

    # Load the abbreviations dictionary
    abbreviations_dict = {}
    with open("abbreviations_dictionary.txt", "r") as file:
        for line in file:
            abbr, full_form = line.strip().split(":")
            abbreviations_dict[abbr.strip()] = full_form.strip()

    target_words_dict = dictionary
    description_words_dict = dictionary

    # Clean and preprocess columns
    df = clean_columns(df)

    print('AFTER CLEAN', df.head())
    df = preprocess_columns(df, abbreviations_dict)

    print(df)

    # Apply analysis
    df = apply_analysis(df, target_words_dict, description_words_dict, abbreviations_dict, datasets_xlsx, DB_or_dataset)

    # Load the FinalFormat from AnalysedColumns
    analysed_df = pd.read_excel(analysed_columns_file_path)
    
    # Merge the FinalFormat into our main dataframe
    df['FinalFormat'] = analysed_df['FinalFormat']

    # Prepare features for machine learning
    df['combined'] = df['CleanedColumn'].fillna('') + ' [SEP] ' + df['Description'].fillna('') 
    df['combined'] += ' [SEP] ' + df['ColumnKeyword'].fillna('') + ' [SEP] ' + df['DescriptionKeyword'].fillna('')

    #df['combined'] += df['ColumnFormat'].fillna('') + ' [SEP] ' + df['DescriptionFormat'].fillna('')
    
    # Load DB headers distribution file
    db_headers = {}
    with open('DBheaders+formats_dict.txt', 'r') as file:
        next(file)  # skip the header line
        for line in file:
            header, count = line.strip().split('\t')
            db_headers[header.lower()] = int(count)

    # Extract features using Custom TF-IDF with vocabulary restricted to DB headers
    vectorizer = CustomTfidfVectorizer(vocabulary=list(db_headers.keys()))
    X = vectorizer.fit_transform(df['combined']).tocsr()

    # Modify TF-IDF scores explicitly based on DB headers
    feature_names = vectorizer.get_feature_names_out()
    weights = np.array([db_headers.get(feat, 1) for feat in feature_names])
  
    # Ensure weights is a 1D array with the same number of elements as X has columns
    if weights.shape[0] != X.shape[1]:
        raise ValueError(f"Number of weights ({weights.shape[0]}) does not match number of features in X ({X.shape[1]})")
    
    # Multiply each column of X by its corresponding weight
    X = X.multiply(weights)

    # Encode the FinalFormat column 
    y = df['FinalFormat'].astype(str)

    # Apply efficient SMOTE
    X_resampled, y_resampled = efficient_smote(X, y)

    print("Class distribution after SMOTE:")
    print(pd.Series(y_resampled).value_counts())

    # Split the resampled data
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

    # List of models to train
    models = [
        (RandomForestClassifier(class_weight='balanced', random_state=42), 'RandomForestClassifier'),
        (LogisticRegression(class_weight='balanced', random_state=42, warm_start=True), 'LogisticRegression'),
        (GradientBoostingClassifier(random_state=42), 'GradientBoostingClassifier'),
        (KNeighborsClassifier(), 'KNeighborsClassifier'),
        (LinearSVC(class_weight='balanced', random_state=42, dual='auto'), 'LinearSVC')
    ]

    # Train, evaluate, and save each model
    for model, model_name in models:
        best_model, report = train_evaluate_save_model(model, model_name, X_train, X_test, y_train, y_test, df, vectorizer, weights, feature_names, output_file_path_text)
        
        # Save classification report
        with open(f'{model_name}_classification_report.txt', 'w') as f:
            f.write(report)

    # Save the vectorizer for future use
    joblib.dump(vectorizer, 'vectorizer.joblib')
    print("Vectorizer saved for future use")

    print(f"Last run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if __name__ == "__main__":
    main()

It started on: 2024-10-03 21:17:36
ds
Using Datasets and Database tables
  index           name database      area  instances  attributes Column  \
0    52           Iris      NaN      Life        150           4    NaN   
1   107           Wine      NaN  Physical        178          13    NaN   
2    92       Spambase      NaN  Computer       4601          57    NaN   
3    45  Heart Disease      NaN      Life        303          75    NaN   
4     2          Adult      NaN    Social      48842          14    NaN   

  Description primary_key foreign_keys  
0         NaN         NaN          NaN  
1         NaN         NaN          NaN  
2         NaN         NaN          NaN  
3         NaN         NaN          NaN  
4         NaN         NaN          NaN  
Original df shape: (1442, 5)
              index    name       area                 Original Column ID  \
0               NaN    Iris       Life     1.       sepal length in cm  1   
1               NaN    Iris       Life      2. 

KeyboardInterrupt: 